# SharedFlow - Podstawy

W tej aplikacji przyjrzymy się zastosowaniu `SharedFlow`.

<img src="https://media4.giphy.com/media/v1.Y2lkPTc5MGI3NjExNTNidmt4dWd6MDV1djVqb2ZlNWhsOXdya2R6YXYwdXNtcG5ia3pqaCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/k80AoHasXQujHAFxTW/giphy.gif" width="200" />

`SharedFlow` jest to specjalny rodzaj strumienia, który można subskrybować przez wiele źródeł, a każdy subskrybent otrzymuje te same emitowane wartości.

Główne cechy `SharedFlow`:
- Współdzielone wartości: Umożliwia emitowanie wartości do wielu obserwatorów. Wszyscy subskrybenci otrzymują te same emitowane wartości, dzięki czemu można synchronizować dane między różnymi komponentami w aplikacji.
- Zachowanie bufora: Może przechowywać określoną ilość ostatnich emitowanych wartości, co pozwala subskrybentom otrzymać te wartości po dołączeniu do strumienia. Można skonfigurować rozmiar bufora przy tworzeniu SharedFlow.

`SharedFlow` jest używane w sytuacjach, gdzie dane muszą być dostarczane do wielu komponentów w aplikacji, na przykład w przypadku synchronizacji stanu między różnymi widokami lub komponentami.

różnice między `StateFlow` a `SharedFlow` :
- Współdzieloność: `StateFlow` reprezentuje strumień, który jest dostępny tylko do odczytu przez jednego subskrybenta. Oznacza to, że jedna instancja StateFlow może mieć tylko jednego aktywnego subskrybenta. Z kolei `SharedFlow` umożliwia wielokrotne subskrybowanie przez różne źródła. Wszyscy subskrybenci otrzymują te same emitowane wartości.
- Zachowanie bufora: `StateFlow` automatycznie przechowuje bieżącą wartość stanu, dzięki czemu nowi subskrybenci otrzymują aktualną wartość od razu po dołączeniu do strumienia. SharedFlow może być skonfigurowane, aby przechowywać określoną ilość ostatnich emitowanych wartości w buforze, które będą dostępne dla nowych subskrybentów po dołączeniu.
- Mutowalność: Oba strumienie umożliwiają mutowanie wartości, ale z różnymi mechanizmami. `StateFlow` używa `MutableStateFlow` jako swojego źródła danych, co umożliwia bezpieczne mutowanie wartości. `SharedFlow` natomiast nie dostarcza bezpośredniego mechanizmu do bezpiecznego mutowania wartości, nowe wartości są emitowane za pomocą metody `emit()` wewnątrz korutyn.

Podsumowując, główna różnica między `StateFlow` a `SharedFlow` polega na sposobie współdzielenia danych i zachowaniu bufora.

Nasza aplikacja będzie prostym licznikiem, w którym po upłynięciu określonego czasu zmienimy wartość `SharedFlow` i zwiększymy stan licznika.

Rozpocznijmy od zaimplementowania `NumberViewModel`

In [ ]:
class NumberViewModel : ViewModel() {

    private var number = 0
    private val _sharedFlow = MutableSharedFlow<Int>(1)
    val sharedFlow: SharedFlow<Int> = _sharedFlow

    init {
        viewModelScope.launch {
            while (true) {
                _sharedFlow.emit(number++)
                delay(500L)
            }
        }
    }
}

- `private var number = 0` - Prywatne pole number, które przechowuje aktualną liczbę.
- `private val _sharedFlow = MutableSharedFlow<Int>(1)` - Jest używany do emitowania wartości typu `Int`. W konstruktorze `MutableSharedFlow` podajemy parametr, który określa pojemność bufora. Oznacza to, że `_sharedFlow` będzie przechowywać tylko jedną, ostatnią wartość do odczytu dla nowych subskrybentów.
- `val sharedFlow: SharedFlow<Int> = _sharedFlow` - Jest udostępnione do subskrybowania - pole tylko do odczytu.
- `viewModelScope.launch { ... }` - `viewModelScope` jest zasięgiem cyklu życia `ViewModel` i zarządza automatycznie odwołaniem wszystkich aktywnych korutyn w momencie zakończenia `ViewModel`.
- `_sharedFlow.emit(number++)` - Emitowanie wartości `number` do `_sharedFlow`.

Zaimplementujmy ui, oraz dodajmy go jako subskrybenta `sharedFlow`

In [ ]:
override fun onCreateView(
    inflater: LayoutInflater, container: ViewGroup?,
    savedInstanceState: Bundle?
): View {
    binding = FragmentNumberBinding.inflate(inflater)

    viewLifecycleOwner.lifecycleScope.launch {
        viewLifecycleOwner.repeatOnLifecycle(Lifecycle.State.STARTED){
            viewModel.sharedFlow.collect{ number ->
                binding.numberText.text = number.toString()
            }
        }
    }

    return binding.root
}

- `viewLifecycleOwner.lifecycleScope.launch { ... }` - Tworzy korutynę w zasięgu życia właściciela widoku. Ta korutyna jest automatycznie anulowana, gdy widok jest zniszczony, co pozwala uniknąć wycieków pamięci.
- `viewLifecycleOwner.repeatOnLifecycle(Lifecycle.State.STARTED) { ... }` - Pozwala na automatyczne powtarzanie bloku kodu. Blok kodu wewnątrz zostanie uruchomiony, gdy widok jest w stanie `STARTED`, a zostanie automatycznie wstrzymany, gdy widok osiągnie stan `STOPPED` lub `DESTROYED`.
- `viewModel.sharedFlow.collect { number -> ... }` - Subskrybuje wartości emitowane przez `sharedFlow`. Za każdym razem, gdy pojawi się nowa wartość, number zostanie zaktualizowane na tę wartość.

Możemy przetestować aplikację

<img src="https://media4.giphy.com/media/v1.Y2lkPTc5MGI3NjExNTNidmt4dWd6MDV1djVqb2ZlNWhsOXdya2R6YXYwdXNtcG5ia3pqaCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/k80AoHasXQujHAFxTW/giphy.gif" width="200" />